# Kryptologie LAB - RSA OAEP

- RSA ist ohne Kenntnis von privatem Schlüssel deterministisch
- Einfache Nachrichten können erraten und überprüft werden
- Padding Protokolle verhindern dies
- Hier: Optimal Asymetric Encryption Padding (OAES)
- Benötigen dafür Hash Funktionen

## Mask Generating Function MGF1

In [1]:
import hashlib
import numpy as np

In [2]:
# Generate mask with given length from a seed and a hash function
# Input: seed als bytearray, Outputlänge, hash funktion
# Output: maskierte bits der länge target_len
def mgf1(seed, target_len, hash_func):
    T = bytearray()
    counter = 0
    while len(T) < target_len:
        counter += 1
        hashable = bytearray(seed)
        for i in range(32, 0, -8):
            hashable.append((counter >> i) & 0xFF)
        T += hash_func(hashable)
    return T[:target_len]

In [3]:
hasher = hashlib.sha1()

def sha1(input):
    hasher.update(input)
    return hasher.digest()


## OAEP Transformation

![OAEP Transformation](OAEP.PNG)

In [4]:
# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [5]:
def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def oaepEncode(M:bytes, n:bytes, hash_func, MGF_func, l = bytearray()):
    lHash = hash_func(l)
    seed = np.random.bytes(len(lHash))
    PS = bytearray(len(n) - len(M) - 2*len(lHash) - 2)

    DB = lHash + PS +  bytes([1]) + M

    maskedDB = byte_xor(DB, MGF_func(seed, len(DB), hash_func))
    maskedSeed = byte_xor(seed, MGF_func(maskedDB, len(seed), hash_func))

    return bytes([0]) + bytes(maskedSeed) + bytes(maskedDB)

In [6]:
n = 17267074447202277029120701952793902504625643746047991963542015468855897842167043414888566049831303080493992468752546359088183287415529571085155018860200517479849599520982888046932224369683506140133500215566532765640525503765127942585822311152218736882121530113737
m = 12345

res = oaepEncode(int_to_bytes(m), int_to_bytes(n), sha1, mgf1)
print(int_from_bytes(res))

55912887364423109442801851565323646537163559779636276447142919459866894349852262505918490786334332701468718291500851482709358427111318564682607445868713245382027977697855334322594998286583037847418854036069099904394596397109550230842148203739381085884400275365


## OAEP Transformation - Inverse

In [7]:
def oaepDecode(input: bytes, hash_func, MGF_func):
    seedLength = len(hash_func(bytearray()))
    maskedSeed = input[1:seedLength+1]
    maskedDB = input[seedLength+1:]
    seed = byte_xor(MGF_func(maskedDB, seedLength, hash_func), maskedSeed)
    DB = byte_xor(MGF_func(seed, len(maskedDB), hash_func), maskedDB)

    # removing lHash
    DB = DB[seedLength:]

    # searching first 1
    index = 0
    while DB[index] == 0:
        index += 1
    M = DB[index+1:]
    return M

In [8]:
decoded = oaepDecode(res, sha1, mgf1)
print(int_from_bytes(decoded))

67098292150268445274862859198758876816076744650820393394590387647496524810249395932611119479465158009303867699209861611205145915736579393396022015325407438671602
